In [18]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

# imports
import numpy as np 
import pandas as pd 
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import signal
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, plot_confusion_matrix
import lightgbm as lgb
from scipy import fftpack
import librosa
from scipy import signal as scisig

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('./drive/My Drive/Colab Notebooks/liverpool-ion-switching/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

./drive/My Drive/Colab Notebooks/liverpool-ion-switching/a-signal-processing-approach-low-pass-filtering.ipynb
./drive/My Drive/Colab Notebooks/liverpool-ion-switching/3-simple-ideas-lb-0-938.ipynb
./drive/My Drive/Colab Notebooks/liverpool-ion-switching/use_sound_feature.ipynb
./drive/My Drive/Colab Notebooks/liverpool-ion-switching/simple_Boosting.ipynb
./drive/My Drive/Colab Notebooks/liverpool-ion-switching/wavenet-with-1-more-feature.ipynb
./drive/My Drive/Colab Notebooks/liverpool-ion-switching/on-markov-chains-and-the-competition-data.ipynb
./drive/My Drive/Colab Notebooks/liverpool-ion-switching/single-model-lgbm-kalman-filter.ipynb
./drive/My Drive/Colab Notebooks/liverpool-ion-switching/shifted-rfc-pipeline.ipynb
./drive/My Drive/Colab Notebooks/liverpool-ion-switching/ion-weighted-voting-ensemble.ipynb
./drive/My Drive/Colab Notebooks/liverpool-ion-switching/music_feature.ipynb
./drive/My Drive/Colab Notebooks/liverpool-ion-switching/score_vote.ipynb
./drive/My Drive/Colab N

In [0]:
B_NUM = 10
B_SIZE = 500000

In [0]:
kal_data = pd.read_csv('./drive/My Drive/Colab Notebooks/liverpool-ion-switching/data/train_clean_batch7_gause.csv')


In [46]:
# batch1 = kal_data[0:B_SIZE]
batch_nums = []
for i in range(B_NUM):
  batches = [i] * B_SIZE
  batch_nums.extend(batches)
kal_data['batch'] = batch_nums
kal_data

,time,signal,open_channels,batch
0,0.0001,-2.760000,0,0
1,0.0002,-2.855700,0,0
2,0.0003,-2.407400,0,0
3,0.0004,-3.140400,0,0
4,0.0005,-3.152500,0,0
...,...,...,...,...
4999995,499.9996,2.932577,7,9
4999996,499.9997,2.711208,7,9
4999997,499.9998,4.529640,8,9
4999998,499.9999,5.652972,9,9


In [0]:
#バターワースフィルタ（バンドストップ）
def bandstop(x, samplerate = 1000000, fp = np.array([4925, 5075]), fs = np.array([4800, 5200])):
    fn = samplerate / 2   # Nyquist frequency
    wp = fp / fn
    ws = fs / fn
    gpass = 1
    gstop = 10.0

    N, Wn = scisig.buttord(wp, ws, gpass, gstop)
    b, a = scisig.butter(N, Wn, "bandstop")
    y = scisig.filtfilt(b, a, x)
    return y

In [48]:
sig_bandstops = []

for i in range(B_NUM):

  batch1 = kal_data[kal_data['batch']==i]
  sig_bandstops.extend(bandstop(batch1['signal']))

kal_data['sig_bandstop'] = sig_bandstops
kal_data

,time,signal,open_channels,batch,sig_bandstop
0,0.0001,-2.760000,0,0,-2.730076
1,0.0002,-2.855700,0,0,-2.825441
2,0.0003,-2.407400,0,0,-2.376835
3,0.0004,-3.140400,0,0,-3.109559
4,0.0005,-3.152500,0,0,-3.121414
...,...,...,...,...,...
4999995,499.9996,2.932577,7,9,2.727882
4999996,499.9997,2.711208,7,9,2.501323
4999997,499.9998,4.529640,8,9,4.314767
4999998,499.9999,5.652972,9,9,5.433318


In [0]:
kal_data_ = pd.read_csv('./drive/My Drive/Colab Notebooks/liverpool-ion-switching/data/train_clean_batch7_gause.csv')

kal_data_['signal'] = kal_data['sig_bandstop'].values

In [0]:
kal_data_.to_csv('./drive/My Drive/Colab Notebooks/liverpool-ion-switching/train_clean_batch7_gause_rem_50hz.csv', index = False)

# TestData

In [0]:
kal_data = pd.read_csv('./drive/My Drive/Colab Notebooks/liverpool-ion-switching/data/test_clean.csv')

In [0]:
B_NUM = 20
B_SIZE = 100000

In [53]:
# batch1 = kal_data[0:B_SIZE]
batch_nums = []
for i in range(B_NUM):
  batches = [i] * B_SIZE
  batch_nums.extend(batches)
kal_data['batch'] = batch_nums
kal_data

,time,signal,batch
0,500.0001,-2.675347,0
1,500.0002,-2.874979,0
2,500.0003,-2.885611,0
3,500.0004,-2.460642,0
4,500.0005,-2.641174,0
...,...,...,...
1999995,699.9996,-2.909200,19
1999996,699.9997,-2.742200,19
1999997,699.9998,-2.828500,19
1999998,699.9999,-2.909200,19


In [54]:
sig_bandstops = []

for i in range(B_NUM):

  batch1 = kal_data[kal_data['batch']==i]
  sig_bandstops.extend(bandstop(batch1['signal']))

kal_data['sig_bandstop'] = sig_bandstops
kal_data

,time,signal,batch,sig_bandstop
0,500.0001,-2.675347,0,-2.705437
1,500.0002,-2.874979,0,-2.905415
2,500.0003,-2.885611,0,-2.916363
3,500.0004,-2.460642,0,-2.491681
4,500.0005,-2.641174,0,-2.672469
...,...,...,...,...
1999995,699.9996,-2.909200,19,-2.866482
1999996,699.9997,-2.742200,19,-2.698942
1999997,699.9998,-2.828500,19,-2.784745
1999998,699.9999,-2.909200,19,-2.864991


In [0]:
kal_data_ = pd.read_csv('./drive/My Drive/Colab Notebooks/liverpool-ion-switching/data/test_clean.csv')

kal_data_['signal'] = kal_data['sig_bandstop'].values

In [0]:
kal_data_.to_csv('./drive/My Drive/Colab Notebooks/liverpool-ion-switching/test_clean_rem_50hz.csv', index = False)